In [2]:
import csv
import os
import glob
import re
from pandas import DataFrame, Series
from openslide import open_slide
from PIL import Image
import timeit
import time
import math

In [57]:
train_paths = ["/scratch/kk4ze/data_svs/valid/Celiac", "/scratch/kk4ze/data_svs/valid/EE", "/scratch/kk4ze/data_svs/valid/Normal", 
              "/scratch/kk4ze/data_svs/train/Celiac", "/scratch/kk4ze/data_svs/train/EE", "/scratch/kk4ze/data_svs/train/Normal"]
images = {}
images_by_folder = {}
for train_path in train_paths:
    images_by_folder[str(train_path)] = []
    files = glob.glob(os.path.join(train_path, '*.svs'))
    for fl in files:
        flbase = os.path.basename(fl)
        flbase_noext = os.path.splitext(flbase)[0]
        images[flbase_noext]=fl
        images_by_folder[str(train_path)].append(flbase)

In [67]:
images.items()

dict_items([('C06-30_04', '/scratch/kk4ze/data_svs/valid/Celiac/C06-30_04.svs'), ('C06-30_03', '/scratch/kk4ze/data_svs/valid/Celiac/C06-30_03.svs'), ('C03-12_01', '/scratch/kk4ze/data_svs/valid/Celiac/C03-12_01.svs'), ('C10-48_03', '/scratch/kk4ze/data_svs/valid/Celiac/C10-48_03.svs'), ('C06-33_03', '/scratch/kk4ze/data_svs/valid/Celiac/C06-33_03.svs'), ('C06-33_02', '/scratch/kk4ze/data_svs/valid/Celiac/C06-33_02.svs'), ('C12-63_02', '/scratch/kk4ze/data_svs/valid/Celiac/C12-63_02.svs'), ('C03-05_04', '/scratch/kk4ze/data_svs/valid/Celiac/C03-05_04.svs'), ('C04-19_03', '/scratch/kk4ze/data_svs/valid/Celiac/C04-19_03.svs'), ('C17-81_04', '/scratch/kk4ze/data_svs/valid/Celiac/C17-81_04.svs'), ('C13-65_02', '/scratch/kk4ze/data_svs/valid/Celiac/C13-65_02.svs'), ('C04-17_05', '/scratch/kk4ze/data_svs/valid/Celiac/C04-17_05.svs'), ('C10-47_01', '/scratch/kk4ze/data_svs/valid/Celiac/C10-47_01.svs'), ('C06-31_01', '/scratch/kk4ze/data_svs/valid/Celiac/C06-31_01.svs'), ('C07-37_05', '/scratc

In [59]:
images_by_folder.keys()

dict_keys(['/scratch/kk4ze/data_svs/valid/Celiac', '/scratch/kk4ze/data_svs/valid/EE', '/scratch/kk4ze/data_svs/valid/Normal', '/scratch/kk4ze/data_svs/train/Celiac', '/scratch/kk4ze/data_svs/train/EE', '/scratch/kk4ze/data_svs/train/Normal'])

In [41]:
path_change_map = {}

for key in list(images_by_folder.keys()):
    temp = key.replace('data_svs', 'data_lowres')
    path_change_map[key] = temp

In [42]:
path_change_map

{'/scratch/kk4ze/data_svs/valid/Celiac': '/scratch/kk4ze/data_lowres/valid/Celiac',
 '/scratch/kk4ze/data_svs/valid/EE': '/scratch/kk4ze/data_lowres/valid/EE',
 '/scratch/kk4ze/data_svs/valid/Normal': '/scratch/kk4ze/data_lowres/valid/Normal',
 '/scratch/kk4ze/data_svs/train/Celiac': '/scratch/kk4ze/data_lowres/train/Celiac',
 '/scratch/kk4ze/data_svs/train/EE': '/scratch/kk4ze/data_lowres/train/EE',
 '/scratch/kk4ze/data_svs/train/Normal': '/scratch/kk4ze/data_lowres/train/Normal'}

In [73]:
for key,value in images_by_folder.items():
        dir=str(key)+'/'+str(value)
        print(dir)

/scratch/kk4ze/data_svs/valid/Celiac/['C06-30_04.svs', 'C06-30_03.svs', 'C03-12_01.svs', 'C10-48_03.svs', 'C06-33_03.svs', 'C06-33_02.svs', 'C12-63_02.svs', 'C03-05_04.svs', 'C04-19_03.svs', 'C17-81_04.svs', 'C13-65_02.svs', 'C04-17_05.svs', 'C10-47_01.svs', 'C06-31_01.svs', 'C07-37_05.svs', 'C14-71_06.svs', 'C03-10_04.svs', 'C12-58_02.svs', 'C13-68_04.svs', 'C04-17_04.svs', 'C07-34_02.svs', 'C12-58_01.svs', 'C17-80_01.svs', 'C07-34_04.svs', 'C16-79_01.svs', 'C04-21_01.svs', 'C03-08_01.svs', 'C12-63_03.svs', 'C14-70_06.svs', 'C10-49_02.svs', 'C03-05_02.svs', 'C03-13_02.svs', 'C03-07_01.svs', 'C03-10_05.svs', 'C03-09_01.svs', 'C06-28_01.svs', 'C07-37_06.svs', 'C06-32_03.svs', 'C10-47_02.svs', 'C10-48_04.svs', 'C11-56_03.svs', 'C11-55_01.svs', 'C17-80_02.svs']
/scratch/kk4ze/data_svs/valid/EE/['34.svs', '130382_6718_001.svs', '87.svs', '130365_6716_001.svs', '130369_6721_001.svs', '72.svs', '111.svs', '79.svs']
/scratch/kk4ze/data_svs/valid/Normal/['N15-23_02.svs', 'N14-13_02.svs', 'N16-

In [30]:
op_slide_img = open_slide('/scratch/kk4ze/Final/Celiac/C13-68_06.svs')

In [31]:
op_slide_img.dimensions

(3309, 7245)

In [29]:
y_break=math.ceil(op_slide_img.dimensions[1]/op_slide_img.dimensions[0])
y_break

3

In [56]:
for count in range(0,y_break):
    x=op_slide_img.dimensions[0]
    if count==0:
        y=0
    else:    
        y=0+op_slide_img.dimensions[1]-(count*x)
    img = op_slide_img.read_region((0,y), 0, (x,x))
    jpg_img = img.convert('RGB')
    jpg_img.save('test_'+str(count)+'.jpg')

In [ ]:
image_path_coord_map = dict((k, []) for k in list(image_patch_data['image_path'].unique()))\n

In [150]:
start_time = time.time()
count = 0
for key,value in image_path_coord_map.items():
    dir=key+'/'+value
    op_slide_img = open_slide(key)
    for val in value:
        x = int(val[0])
        y = int(val[1])
        img = op_slide_img.read_region((x,y), 0, (1000,1000))
        img = img.resize(width_height, Image.ANTIALIAS)
        jpg_img = img.convert('RGB')
        image_name = key.split('/')[-1]
        image_name = image_name.split('.')[0]
        jpg_img.save(path_change_map[key] + '/' + image_name + '_' + str(x) + '_' + str(y) + '.jpg')
    print(key + ' Saved!')
    count+=1
    print(count/451)
    stop = time.time()
    print(stop-start_time)
stop = time.time()
print(stop-start_time)

../data/valid/Celiac/C07-37_06.svs Saved!
0.0022172949002217295
205.2351496219635


KeyboardInterrupt: 